In [36]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
import numpy as np

In [37]:
#data processing

TRAINING_DIR = ('D:\\1_Projects_at_UC_Davis\\0_Research1_ML_Weidi\\c-1\\train')
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

VALIDATION_DIR = ('D:\\1_Projects_at_UC_Davis\\0_Research1_ML_Weidi\\c-1\\test')
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [38]:
train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(198,198),
	class_mode='categorical',
  batch_size=2
)

Found 3600 images belonging to 2 classes.


In [39]:
validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(198,198),
	class_mode='categorical',
  batch_size=2
)

Found 400 images belonging to 2 classes.


In [40]:
class VGG16(object):

	""" Implementation of VGG16 network """

	def __init__(self, x, keep_prob, num_classes):

		"""Create the graph of the AlexNet model.
        Args:
            x: Placeholder for the input tensor.
            keep_prob: Dropout probability.
            num_classes: Number of classes in the dataset.
        """

        # Parse input arguments into class variables
		self.X = x
		self.NUM_CLASSES = num_classes
		self.KEEP_PROB = keep_prob

		# Call the create function to build the computational graph of AlexNet
		self.create()

	def create(self):

		"""Create the network graph."""

		conv1_1 = conv_layer(self.X, 64, 'conv1_1')
		conv1_2 = conv_layer(conv1_1, 64, 'conv1_2')
		pool1 = max_pool(conv1_2, 'pool1')

		conv2_1 = conv_layer(pool1, 128, 'conv2_1')
		conv2_2 = conv_layer(conv2_1, 128, 'conv2_2')
		pool2 = max_pool(conv2_2, 'pool2')

		conv3_1 = conv_layer(pool2, 256, 'conv3_1')
		conv3_2 = conv_layer(conv3_1, 256, 'conv3_2')
		conv3_3 = conv_layer(conv3_2, 256, 'conv3_3')
		pool3 = max_pool(conv3_3, 'pool3')

		conv4_1 = conv_layer(pool3, 512, 'conv4_1')
		conv4_2 = conv_layer(conv4_1, 512, 'conv4_2')
		conv4_3 = conv_layer(conv4_2, 512, 'conv4_3')
		pool4 = max_pool(conv4_3, 'pool4')

		conv5_1 = conv_layer(pool4, 512, 'conv5_1')
		conv5_2 = conv_layer(conv5_1, 512, 'conv5_2')
		conv5_3 = conv_layer(conv5_2, 512, 'conv5_3')
		pool5 = max_pool(conv5_3, 'pool5')

		flattened = tf.reshape(pool5, [-1, 1 * 1 * 512])
		fc6 = fc_layer(flattened, 1 * 1 * 512, 4096, name = 'fc6')
		dropout6 = dropout(fc6, self.KEEP_PROB)

		fc7 = fc_layer(dropout6, 4096, 4096, name = 'fc7')
		dropout7 = dropout(fc7, self.KEEP_PROB)

		self.fc8 = fc_layer(dropout7, 4096, self.NUM_CLASSES, relu = False, name = 'fc8')



def conv_layer(x, num_filters, name, filter_height = 3, filter_width = 3,
	stride = 1, padding = 'SAME'):

	"""Create a convolution layer."""
	
	# Get number of input channels
	input_channels = int(x.get_shape()[-1])

	with tf.variable_scope(name) as scope:

		# Create tf variables for the weights and biases of the conv layer
		W = tf.get_variable('weights', shape = [filter_height, filter_width, input_channels, num_filters],
			initializer = tf.random_normal_initializer(mean = 0.0, stddev = 0.01))

		b = tf.get_variable('biases', shape = [num_filters], initializer = tf.constant_initializer(0.0))

		# Perform convolution.
		conv = tf.nn.conv2d(x, W, strides = [1, stride, stride, 1], padding = padding)
		# Add the biases.
		z = tf.nn.bias_add(conv, b)
		# Apply ReLu non linearity.
		a = tf.nn.relu(z)

		return a

def fc_layer(x, input_size, output_size, name, relu = True):

	"""Create a fully connected layer."""
	
	with tf.variable_scope(name) as scope:

		# Create tf variables for the weights and biases.
		W = tf.get_variable('weights', shape = [input_size, output_size],
			initializer = tf.random_normal_initializer(mean = 0.0, stddev = 0.01))

		b = tf.get_variable('biases', shape = [output_size], initializer = tf.constant_initializer(0.0))

		# Matrix multiply weights and inputs and add biases.
		z = tf.nn.bias_add(tf.matmul(x, W), b)

		if relu:
			# Apply ReLu non linearity.
			a = tf.nn.relu(z)
			return a

		else:
			return z

def max_pool(x, name, filter_height = 2, filter_width = 2,
	stride = 2, padding = 'VALID'):

	"""Create a max pooling layer."""

	return tf.nn.max_pool(x, ksize = [1, filter_height, filter_width, 1],
		strides = [1, stride, stride, 1], padding = padding, name = name)

def dropout(x, keep_prob):

	"""Create a dropout layer."""

	return tf.nn.dropout(x, keep_prob = keep_prob)

In [41]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(198, 198, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])


model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 196, 196, 64)      1792      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 98, 98, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 96, 96, 64)        36928     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 48, 48, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 46, 46, 128)       73856     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 23, 23, 128)     

In [42]:
def recall_m(y_true, y_pred):
    true_positive = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positive / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positive = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision_result = true_positive / (predicted_positives + K.epsilon())
    return precision_result

def f1_score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    f1_score = 2*((precision*recall)/(precision+recall+K.epsilon()))
    return f1_score

In [43]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy',recall_m, precision_m, f1_score])


In [44]:
history = model.fit(train_generator, epochs=50, steps_per_epoch=30, validation_data = validation_generator, verbose = 1, validation_steps=3)


Epoch 1/50
30/30 [==============================] - 8s 212ms/step - loss: 0.9980 - accuracy: 0.5667 - recall_m: 0.5667 - precision_m: 0.5667 - f1_score: 0.5667 - val_loss: 0.7045 - val_accuracy: 0.3333 - val_recall_m: 0.3333 - val_precision_m: 0.3333 - val_f1_score: 0.3333
Epoch 2/50
30/30 [==============================] - 6s 209ms/step - loss: 0.7188 - accuracy: 0.6167 - recall_m: 0.6167 - precision_m: 0.6167 - f1_score: 0.6167 - val_loss: 0.6051 - val_accuracy: 1.0000 - val_recall_m: 1.0000 - val_precision_m: 1.0000 - val_f1_score: 1.0000
Epoch 3/50
30/30 [==============================] - 6s 214ms/step - loss: 0.7108 - accuracy: 0.7500 - recall_m: 0.7500 - precision_m: 0.7500 - f1_score: 0.7500 - val_loss: 0.7608 - val_accuracy: 0.8333 - val_recall_m: 0.8333 - val_precision_m: 0.8333 - val_f1_score: 0.8333
Epoch 4/50
30/30 [==============================] - 6s 208ms/step - loss: 0.7161 - accuracy: 0.8167 - recall_m: 0.8167 - precision_m: 0.8167 - f1_score: 0.8167 - val_loss: 0.4073

Epoch 31/50
30/30 [==============================] - 6s 210ms/step - loss: 0.3140 - accuracy: 0.9000 - recall_m: 0.9000 - precision_m: 0.9000 - f1_score: 0.9000 - val_loss: 1.6768 - val_accuracy: 0.6667 - val_recall_m: 0.6667 - val_precision_m: 0.6667 - val_f1_score: 0.6667
Epoch 32/50
30/30 [==============================] - 6s 208ms/step - loss: 0.4529 - accuracy: 0.8833 - recall_m: 0.8833 - precision_m: 0.8833 - f1_score: 0.8833 - val_loss: 0.0748 - val_accuracy: 1.0000 - val_recall_m: 1.0000 - val_precision_m: 1.0000 - val_f1_score: 1.0000
Epoch 33/50
30/30 [==============================] - 6s 213ms/step - loss: 0.2750 - accuracy: 0.9333 - recall_m: 0.9333 - precision_m: 0.9333 - f1_score: 0.9333 - val_loss: 0.0057 - val_accuracy: 1.0000 - val_recall_m: 1.0000 - val_precision_m: 1.0000 - val_f1_score: 1.0000
Epoch 34/50
30/30 [==============================] - 6s 203ms/step - loss: 0.7131 - accuracy: 0.9000 - recall_m: 0.9000 - precision_m: 0.9000 - f1_score: 0.9000 - val_loss: 0.